In [1]:
from dataloaders import *
from network import *
from metrics import *

In [2]:
multiomics_clean = pd.read_csv("../data_csv/multiomics_svd_100x100.csv")
# multiomics_clean = pd.read_csv("../data_csv/multiomics_auc_500.csv")
print(multiomics_clean.shape)
target_tags = pd.Index(['2D10-LTR5', '2D10-Tat', '2D10-Vpu', '2D10-Env', '2D10-d2EGFP',
       '2D10-LTR3', 'Target'])
target_tag = pd.Index(['Target'])
other_columns = multiomics_clean.columns.drop(target_tags).drop(pd.Index(["dataset"]))

multiomics_clean['Target'] = (multiomics_clean["Target"] > 0).astype(int)
print(multiomics_clean['Target'])


pd_train = multiomics_clean.loc[multiomics_clean["dataset"] == "train"]
pd_test = multiomics_clean.loc[multiomics_clean["dataset"] == "test"]

(3900, 4134)
0       0
1       0
2       1
3       0
4       0
       ..
3895    1
3896    0
3897    0
3898    0
3899    0
Name: Target, Length: 3900, dtype: int32


In [3]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4)
index_splits = kf.split(pd_train)

for train_index, test_index in index_splits:
    pd_train_split, pd_val_split = pd_train.iloc[train_index], pd_train.iloc[test_index]

In [4]:
other_columns = pd_train_split.columns.drop(target_tags).drop(pd.Index(["dataset"]))
print(len(other_columns))

4126


In [5]:
train_dataloader = get_dataloaders(pd_train_split, other_columns, utility_tag=target_tag, batch_size = 32)

val_dataloader = get_dataloaders(pd_val_split, other_columns, utility_tag=target_tag, batch_size = 32)

test_dataloader = get_dataloaders(pd_test, other_columns, utility_tag=target_tag, batch_size = 32)

In [6]:
for a in train_dataloader:
    print(a[1].shape)

torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])


In [7]:
input_dim = 500
hidden_dim = 128
output_dim = 2

device = torch.device('cuda:0')
model = MultiLayerPerceptron(input_dim, hidden_dim, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

torch.cuda.device_count()

1

In [8]:
epochs = 10

model.train()
for epoch in range(epochs):
    losses = []
    for batch_num, input_data in enumerate(train_dataloader):
        optimizer.zero_grad()
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device).long()

        out1 = model(x)
        output = nn.Softmax(dim=1)(out1)
        loss = criterion(output, y.squeeze())
        loss.backward()
        losses.append(loss.item())

        optimizer.step()

        if batch_num % 40 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
model.eval()
outputs = []
trues = []
for batch_num, input_data in enumerate(test_dataloader):
    x, y = input_data
    [trues.append(y.cpu().detach().numpy()[i]) for i in range(y.shape[0])]
    x = x.to(device).float()
    y = y.to(device).float()

    output = model(x).cpu().detach().numpy()
    cat = np.argmax(output, axis=1)

    [outputs.append(cat[i]) for i in range(output.shape[0])]
y_hat = np.array(outputs)
y_true = np.array(trues)

In [ ]:
binary_metrics(y_true, y_hat, binary=True)